<a href="https://colab.research.google.com/github/ivorykimber/Groupme-Bot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to load and train on the data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/ACML\ Onboarding\ Project\ (Team\ FroSoThots)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ACML Onboarding Project (Team FroSoThots)


In [0]:
# We can just copy the libraries from the other workshop code

import numpy as np                   # for matrices
import torch                         # PyTorch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as utils
import time
import copy

# other imports for the purposes of this notebook
from google.colab import files
import io
import util

In [0]:
# CONSTANTS
BATCH_SIZE = 100

In [0]:
import os

In [0]:
os.getcwd()

'/content/drive/My Drive/ACML Onboarding Project (Team FroSoThots)'

In [0]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
## actual LA dataset ##
imgs = torch.Tensor(np.load('selected_imgs.npy'))
labels = torch.Tensor(np.load('labels.npy'))

In [0]:
#imgs.size()
imgs = imgs.view(1472, 3, 256, 256)

In [0]:
LA_dataset = utils.TensorDataset(imgs, labels)

In [0]:
train_size = int(0.9 * len(LA_dataset))
val_size = len(LA_dataset) - train_size
LA_train, LA_val = torch.utils.data.random_split(LA_dataset, [train_size, val_size])

In [0]:
train_sampler = SubsetRandomSampler(np.arange(train_size, dtype=np.int64))
val_sampler = SubsetRandomSampler(np.arange(val_size, dtype=np.int64))

In [0]:
class ConvolutionalNeuralNetwork(torch.nn.Module):
  def __init__(self):
    super(ConvolutionalNeuralNetwork, self).__init__()
    
    self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
    self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    
    self.fc1 = torch.nn.Linear(18 * 16 * 16, 1)
    
    self.fc2 = torch.nn.Linear(64, 1)
  def forward(self, x):     
    x = F.relu(self.conv1(x))
    print("after x: {}".format(x.size()))
    x = self.pool(x)
    x = x.view(-1, 18 * 16 *16)
    x = F.relu(self.fc1(x))
    x = self.fc1(x)
    print("final x: {}".format(x.size()))
    return x

In [0]:
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(LA_train, batch_size=batch_size,
                                               sampler=train_sampler, num_workers=2)
    return train_loader

In [0]:
val_loader = torch.utils.data.DataLoader(LA_val, batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=2)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return loss, optimizer

In [0]:
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    best_model_wts = copy.deepcopy(net.state_dict())
    best_acc = 0.0
    
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    training_start_time = time.time()
    
    
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        running_corrects = 0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        epoch_size = 0;
        
        for i, data in enumerate(train_loader, 0):
            
            epoch_size+=1;
            inputs, labels = data
            
            inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            print(inputs.size())
            print(outputs.size())
            print(labels.size())
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            running_loss +=  loss_size.data
            running_corrects += torch.sum(preds == labels.data);
            total_train_loss += loss_size.data
            
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                running_loss = 0.0
                start_time = time.time()
        epoch_loss = running_loss / epoch_size
        epoch_acc = running_corrects.double() / epoch_size
            
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            inputs, labels = Variable(inputs), Variable(labels)
            
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data

        if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(net.state_dict())
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

    net.load_state_dict(best_model_wts)
    return net;


In [0]:
inputs = [x for x,y in get_train_loader(10)]

In [0]:
inputs[0].size()

torch.Size([10, 3, 32, 32])

In [0]:
#CNN = torchvision.models.resnet18(pretrained=False)
CNN = ConvolutionalNeuralNetwork()
CNN = trainNet(CNN, batch_size=BATCH_SIZE, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 100
epochs= 5
learning_rate= 0.001
after x: torch.Size([100, 18, 256, 256])
torch.Size([100, 3, 256, 256])
torch.Size([6400, 1])
torch.Size([100])


ValueError: ignored

In [0]:
model_conv = torchvision.models.resnet18(pretrained=False)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 74.7MB/s]


In [0]:
type(model_conv)

torchvision.models.resnet.ResNet

In [0]:
CNN = ConvolutionalNeuralNetwork()

In [0]:
CNN

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [0]:

for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)